In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
style.available

In [ ]:
style.use("fivethirtyeight")

# Basic Exploration

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
train.drop("row_id", axis=1, inplace=True)

In [ ]:
train.drop_duplicates(inplace=True)
train.shape

In [ ]:
x, y = train.drop("target", axis=1), train["target"]

In [ ]:
x.head()

In [ ]:
y

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(y)
plt.show()

In [ ]:
pip install sweetviz

In [ ]:
import sweetviz

In [ ]:
#myreport = sweetviz.analyze([x, 'x'], pairwise_analysis='on')

In [ ]:
#myreport.show_html()

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
y = enc.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xval, ytrain, yval = train_test_split(x, y, test_size=0.07)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(min_samples_split=15)

In [ ]:
%time rf.fit(xtrain,ytrain)

In [ ]:
from sklearn.preprocessing import RobustScaler
ss = RobustScaler()

In [ ]:
xtrain = pd.DataFrame(ss.fit_transform(xtrain), columns=xtrain.columns)
xval = pd.DataFrame(ss.transform(xval), columns=xval.columns)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()

In [ ]:
xtrain_pca = pca.fit_transform(xtrain)
xval_pca = pca.transform(xval)

In [ ]:
xtrain_pca.shape

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(x=range(1,287), height=pca.explained_variance_ratio_, color="green")
plt.show()

In [ ]:
xtrain_pca = xtrain_pca[:, :35]
xval_pca = xval_pca[:, :35]

In [ ]:
xtrain_pca.shape

In [ ]:
%time rf.fit(xtrain_pca, ytrain)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
def evaluate(x,y):
    ypred = rf.predict(x)
    print(classification_report(y, ypred))

In [ ]:
evaluate(xtrain_pca, ytrain)

In [ ]:
evaluate(xval_pca, yval)

In [ ]:
x = pca.fit_transform(ss.fit_transform(x))[:, :35]
rf.fit(x,y)

In [ ]:
test.head()

In [ ]:
rid = test["row_id"]

In [ ]:
rid

In [ ]:
test.drop("row_id", axis=1, inplace=True)

In [ ]:
test = pca.transform(ss.transform(test))[:, :35]

In [ ]:
ysub = rf.predict(test)

In [ ]:
ysub = enc.inverse_transform(ysub)

In [ ]:
sub = pd.DataFrame({"row_id":rid, "target":ysub}).set_index("row_id")

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=True)